#Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    !pip install wandb --upgrade -qq
    !pip install torchmetrics -qq
    !pip install torchtext --upgrade -qq
    !pip install swifter -qq
    !pip install torch_lr_finder -qq
    !pip install --upgrade gensim -qq
    !pip install -U spacy -qq

    basepath = '/content/drive/MyDrive/NLP_UTD/Assignments/Assignment_7'
    sys.path.append('/content/drive/MyDrive/NLP_UTD/custom-functions')
else:
    basepath = '/home/harpreet/Insync/google_drive_shaannoor/data'
    sys.path.append(
        '/home/harpreet/Insync/google_drive_shaannoor/data/custom-functions')

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 kB 31.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.3 MB/s eta 0:00:00


In [3]:
from pathlib import Path
base_folder = Path(basepath)
data_folder = base_folder
model_folder = base_folder
custom_functions = '/content/drive/MyDrive/NLP_UTD/custom-functions'
# sys.path.append(str(custom_functions))
sys.path

['/content',
 '/env/python',
 '/usr/lib/python39.zip',
 '/usr/lib/python3.9',
 '/usr/lib/python3.9/lib-dynload',
 '',
 '/usr/local/lib/python3.9/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.9/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/NLP_UTD/custom-functions']

In [4]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [5]:
# Importing the necessary libraries
import torch
import torch.nn as nn
import numpy as np
import wandb
import pandas as pd
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
import pandas as pd
from torchtext.vocab import vocab
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torchtext.vocab import vocab
import torchmetrics
from torchmetrics.classification import Accuracy
import random
from collections import Counter
from types import SimpleNamespace
from functools import partial
from sklearn.metrics import accuracy_score, f1_score, hamming_loss
from Trainer_v4 import *
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from data_preparation_imdb_rev0 import *
from ff_3_layer_sequential_model import MLPCustom
from  torchmetrics.functional.classification import multilabel_f1_score, multilabel_hamming_distance
from torchmetrics import HammingDistance

from torchmetrics.classification import MultilabelAccuracy, MultilabelHammingDistance
import torch.nn as nn
import torch.nn.functional as F
import joblib
import swifter

from types import SimpleNamespace
from functools import partial
from data_preparation_HW7 import *
from ff_sequential_model_v1 import *
import random
import spacy
import textwrap
import joblib
from pathlib import Path
import numpy as np
import gensim
import gensim.downloader as loader
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import itertools
import custom_preprocessor_mod as cp

/usr/local/lib/python3.9/dist-packages/torch_lr_finder/lr_finder.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
import os
os.cpu_count()

2

In [7]:
print(f'spacy-version: {spacy.__version__}  || gensim-version: { gensim.__version__}')


spacy-version: 3.5.2  || gensim-version: 4.3.1


In [8]:
!python3 --version

Python 3.9.16


In [9]:
!python -m spacy download en_core_web_lg -qq

2023-04-25 22:33:43.065781: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [10]:
if gensim.models.doc2vec.FAST_VERSION > -1:
    print("Fast gensim version is being used.")
else:
    print("Slow gensim version is being used.")

Fast gensim version is being used.


# Load through Joblib

##Data File 1

In [ ]:
df = joblib.load('/content/drive/MyDrive/NLP_UTD/Assignments/Assignment_7/df_multilabel_hw_cleaned.joblib')
df.head()

,cleaned_text,Tags,Tag_Number
0,asp query stre dropdown webpage follow control...,c# asp.net,"[0, 9]"
1,run javascript code server java code want run ...,java javascript,"[1, 3]"
2,linq sql throw exception row find change hi li...,c# asp.net,"[0, 9]"
3,run python script php server run nginx web ser...,php python,"[2, 7]"
4,advice write function m try write function res...,javascript jquery,"[3, 5]"


In [ ]:
df['Tag_Number'] = df['Tag_Number'].apply(ast.literal_eval)
print(df['Tag_Number'].dtype)
print(type(df['Tag_Number'][0][1]))

object
<class 'int'>


In [ ]:
mlb = MultiLabelBinarizer()
tags_encoded = mlb.fit_transform(df['Tag_Number'])
df['Tags_Encoded'] = pd.Series(tags_encoded.tolist())
df.head()

,cleaned_text,Tags,Tag_Number,Tags_Encoded
0,asp query stre dropdown webpage follow control...,c# asp.net,"[0, 9]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
1,run javascript code server java code want run ...,java javascript,"[1, 3]","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0]"
2,linq sql throw exception row find change hi li...,c# asp.net,"[0, 9]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
3,run python script php server run nginx web ser...,php python,"[2, 7]","[0, 0, 1, 0, 0, 0, 0, 1, 0, 0]"
4,advice write function m try write function res...,javascript jquery,"[3, 5]","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0]"


#Train, Valid & Test split

In [ ]:
x_train, x_val_test, y_train, y_val_test = train_test_split(df['cleaned_text'], df['Tags_Encoded'], test_size = 0.4, random_state = 42)
x_valid, x_test, y_valid, y_test = train_test_split(x_val_test, y_val_test, test_size = 0.5, random_state = 42)

# Print the shapes of the resulting datasets
print("Train set shape:", x_train.shape, y_train.shape)
print("Validation set shape:", x_valid.shape, y_valid.shape)
print("Test set shape:", x_test.shape, y_test.shape)

Train set shape: (28456,) (28456,)
Validation set shape: (9485,) (9485,)
Test set shape: (9486,) (9486,)


##Data File 2

In [11]:
df_raw = joblib.load('/content/drive/MyDrive/NLP_UTD/Assignments/Assignment_7/df_raw_small_hw.joblib')
df_raw.head()

,Unnamed: 0,Id,Title,Body
3595496,1220704,7254900,Identify un-activated servers,<p>I've been on a bit of a roll this week and ...
1929804,4905250,4905251,How to abort the loading of an external HTML r...,"<p>I'm writing a JavaScript widget, which is i..."
2513704,138912,6173108,Xcode: how do you make it to where clicking on...,<p>I know how to make a button in Xcode with I...
3794169,1419377,7453573,Common WQL Monitoring Queries,<p>What WQL queries would you use for monitori...
4341677,1966885,8001081,Loading Native Managed and C++ DLL within IIS ...,<p>I have WCF service developed in C# for with...


In [12]:
# df_raw_small = df_raw.sample(frac = 0.1, replace = True, random_state = 1)
x = df_raw['Body'].values
len(x)

438813

In [ ]:
x[0:3]

array(['<p>I\'ve been on a bit of a roll this week and have deployed around 30 fresh installs of Windows Server 2008 R2 using our Datacentre license, inside a VMWare environment.</p>\n\n<p>However, what I\'ve gone and forgotten as part of the setup routine is to activate the servers.</p>\n\n<p>The last thing I want is next month to find out that our clients can\'t log into our terminal servers because I\'ve forgotten to activate them, or for them to see "You have been a victim of software piracy" pop up on their desktop.</p>\n\n<p>Is there any script/dirty hacks that I can use to quickly scan the network for un-activated machines? I\'m guessing it will probably be a powershell script stuck into a GPO, but that\'s as far as my imagination can take me at the moment.</p>\n',
       "<p>I'm writing a JavaScript widget, which is intended for use on other website and loads additional script files from my own web site (which basically means no XHR due to cross-domain limitations). Now I want 

In [13]:
x_cleaned = cp.SpacyPreprocessor('en_core_web_lg', lemmatize=False, remove_stop=False,).transform(x)
x_cleaned_text_file_HW7_task1 = data_folder/'x_cleaned_text_file_HW7_task1.pkl'
joblib.dump(x_cleaned, x_cleaned_text_file_HW7_task1)
x_cleaned_text_file_HW7_task1 = joblib.load(x_cleaned_text_file_HW7_task1)

In [ ]:
x_cleaned[0:3]

["i 've been on a bit of a roll this week and have deployed around 30 fresh installs of windows server 2008 r2 using our datacentre license inside a vmware environment however what i 've gone and forgotten as part of the setup routine is to activate the servers the last thing i want is next month to find out that our clients ca n't log into our terminal servers because i 've forgotten to activate them or for them to see you have been a victim of software piracy pop up on their desktop is there any script dirty hacks that i can use to quickly scan the network for un activated machines i 'm guessing it will probably be a powershell script stuck into a gpo but that 's as far as my imagination can take me at the moment",
 "i 'm writing a javascript widget which is intended for use on other website and loads additional script files from my own web site which basically means no xhr due to cross domain limitations now i want to handle failures gracefully if one script fails to load i want to 

In [14]:
sentences = [s.split() for s in x_cleaned_text_file_HW7_task1]
cbow_model_HW7 = gensim.models.Word2Vec(sentences, epochs=10, vector_size=300, min_count=5, window=5, workers=1, sg=0)

In [15]:
cbow_model_HW7.wv.save(str(base_folder/'cbow_model_AJ.bin'))
cbow_vectors_HW7 = KeyedVectors.load(str(base_folder/'cbow_model_AJ.bin'))

In [ ]:
cbow_vectors_HW7['this']

array([ 0.14728409,  1.8955944 ,  0.94737846, -0.35931456, -0.6872438 ,
       -0.09094121,  2.9345434 ,  4.7620482 ,  1.0938944 , -1.7128952 ,
       -1.8256688 , -1.6340438 ,  1.6850901 , -1.7207308 , -1.0479951 ,
       -1.1460652 ,  3.3883076 , -1.6060137 ,  0.6820141 , -0.42677334,
        0.43412474, -1.5746964 , -0.2054762 , -0.26916727, -0.01717711,
       -0.35796854, -1.9035767 ,  2.1411667 ,  0.5810792 , -1.9437028 ,
        0.58190644, -2.3619242 ,  0.6124924 , -1.5502583 ,  0.8544821 ,
        2.9893086 ,  1.1154089 , -0.32074538, -1.129683  ,  0.6540766 ,
       -3.1548355 ,  0.80292594,  0.08148962, -0.7256743 , -1.0951047 ,
       -1.3348442 ,  0.40485114,  1.6367387 , -1.9290782 , -1.717348  ,
        0.07781211, -0.4010835 , -1.1859248 ,  1.1473062 , -1.5916113 ,
       -0.39004713, -2.3102376 ,  0.1906195 , -1.8371776 , -0.8269194 ,
       -1.3759426 , -0.94189256,  0.5177538 , -0.2989076 , -1.7258736 ,
       -1.6293263 ,  1.5381188 ,  0.7128592 , -2.2488856 , -2.52

In [ ]:
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
texlive-fonts-recommended is already the newest version (2019.20200218-1).
texlive-plain-generic is already the newest version (2019.202000218-1).
texlive-xetex is already the newest version (2019.20200218-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!jupyter nbconvert --to pdf /content/drive/MyDrive/NLP_UTD/Assignments/Assignment_7/JyotsnaNagaprasad_Ambekar_HW7_task1.ipynb 

[NbConvertApp] Converting notebook /content/drive/MyDrive/NLP_UTD/Assignments/Assignment_7/JyotsnaNagaprasad_Ambekar_HW7_task1.ipynb to pdf
[NbConvertApp] Writing 75717 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 71098 bytes to /content/drive/MyDrive/NLP_UTD/Assignments/Assignment_7/JyotsnaNagaprasad_Ambekar_HW7_task1.pdf
